In [296]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gp
import numpy as np
pd.options.display.max_columns = 999

In [297]:
store = pd.HDFStore('C:/Users/ramu225/University of Kentucky/Erhardt, Greg - SF-TNC/Runs/sfdata_wrangler2/out/muni_metro_monthly_ts.h5')

In [202]:
store.items

<bound method HDFStore.items of <class 'pandas.io.pytables.HDFStore'>
File path: C:/Users/ramu225/University of Kentucky/Erhardt, Greg - SF-TNC/Runs/sfdata_wrangler2/out/muni_metro_monthly_ts.h5
/rs_day                          frame_table  (typ->appendable,nrows->7900,ncols->58,indexers->[index],dc->[MONTH,DOW,AGENCY_ID,ROUTE_SHORT_NAME,DIR,SEQ,NUMDAYS,OBSDAYS,TRIP_STOPS,OBS_TRIP_STOPS,IMP_TRIP_STOPS,WGT_TRIP_STOPS,STOP_ID,ROUTE_LONG_NAME,ROUTE_TYPE,TRIP_HEADSIGN,HEADWAY_S,FARE,STOPNAME,STOPNAME_AVL,STOP_LAT,STOP_LON,EOL,SOL,TIMEPOINT,ARRIVAL_TIME_DEV,DEPARTURE_TIME_DEV,DWELL_S,DWELL,RUNTIME_S,RUNTIME,TOTTIME_S,TOTTIME,SERVMILES_S,SERVMILES,RUNSPEED_S,RUNSPEED,TOTSPEED_S,TOTSPEED,ONTIME5,ON,OFF,LOAD_ARR,LOAD_DEP,PASSMILES,PASSHOURS,WAITHOURS,FULLFARE_REV,PASSDELAY_DEP,PASSDELAY_ARR,RDBRDNGS,DOORCYCLES,WHEELCHAIR,BIKERACK,CAPACITY,VC,CROWDED,CROWDHOURS])     
/rs_tod                          frame_table  (typ->appendable,nrows->52458,ncols->59,indexers->[index],dc->[MONTH,DOW,TOD,AGENC

In [298]:
df09 = store.select('rs_day',where = "'MONTH' == ['2009-10-01T00:00:00.000000000','2009-11-01T00:00:00.000000000', '2009-12-01T00:00:00.000000000']")

In [299]:
df16 = store.select('rs_day',where = "'MONTH' == ['2016-10-01T00:00:00.000000000', '2016-11-01T00:00:00.000000000', '2016-12-01T00:00:00.000000000']")

In [205]:
df09.MONTH.unique()

array(['2009-10-01T00:00:00.000000000', '2009-11-01T00:00:00.000000000',
       '2009-12-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [206]:
df16.MONTH.unique()

array(['2016-10-01T00:00:00.000000000', '2016-11-01T00:00:00.000000000',
       '2016-12-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [300]:
riders09 = pd.read_csv('E:/Transit-Casa-Alex/MUNI Rail/Input/Raw Ridership Data/MUNI Rail Ridership_Post_Fix_2009.csv')

In [301]:
riders16 = pd.read_csv('E:/Transit-Casa-Alex/MUNI Rail/Input/Raw Ridership Data/MUNI_Rail_Ridership_Post_Fix_2016.csv')

In [209]:
riders16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 15 columns):
ROUTE                             2127 non-null object
DIRECTION                         2127 non-null object
STOP NAME                         2127 non-null object
STOP ID                           2127 non-null int64
STOP SEQ                          2127 non-null int64
Time Period                       2127 non-null object
 Total Boardings                  2127 non-null object
 Total Alightings                 2127 non-null object
 Total Train Load                 2127 non-null object
Number of Trips in Time Period    2127 non-null int64
 Average Boardings                2127 non-null object
 Average Alightings               2127 non-null object
 Average Train Load               2127 non-null object
Stop Latitude                     2127 non-null float64
Stop Longitude                    2127 non-null float64
dtypes: float64(2), int64(3), object(10)
memory usage: 249.3+ KB


In [210]:
riders16.columns

Index(['ROUTE', 'DIRECTION', 'STOP NAME', 'STOP ID', 'STOP SEQ', 'Time Period',
       ' Total Boardings ', ' Total Alightings ', ' Total Train Load ',
       'Number of Trips in Time Period', ' Average Boardings ',
       ' Average Alightings ', ' Average Train Load ', 'Stop Latitude',
       'Stop Longitude'],
      dtype='object')

In [302]:
riders16[' Average Boardings '] = riders16[' Average Boardings '].convert_objects(convert_numeric = True)
riders16[' Average Alightings '] = riders16[' Average Alightings '].convert_objects(convert_numeric = True)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [212]:
riders09.head()

,Route,STOP,Boardings,Alightings,DIR,STOP ID,STOP NAME
0,J-CHURCH,Green Div Yard,0,412,0,5448,SAN JOSE AVE & GENEVA AVE
1,J-CHURCH,S.Jose&Ocean,3,172,0,6274,San Jose Ave & Ocean Ave
2,J-CHURCH,ROW&Liberty,12,95,0,6221,Right Of Way/Liberty St
3,J-CHURCH,ROW&21st,13,148,0,6216,Right Of Way/21st St
4,J-CHURCH,Glen Park Sta,18,146,0,4788,San Jose Ave/Glen Park Station


In [213]:
riders09.columns

Index(['Route', 'STOP', 'Boardings', 'Alightings', 'DIR', 'STOP ID',
       'STOP NAME'],
      dtype='object')

In [303]:
riders09.columns = ['Route', 'STOP', 'Boardings', 'Alightings', 'DIR', 'STOP_ID',
       'STOP NAME']

In [215]:
riders09.Route.unique()

array(['J-CHURCH', 'KT-INGLESIDE/THIRD', 'L-TARAVAL', 'M-OCEAN VIEW',
       'N-JUDAH'], dtype=object)

In [304]:
def set_short_name09(row):
    if row['Route'] == 'J-CHURCH':
        row['ROUTE_SHORT_NAME'] = 'J'
    elif row['Route'] == 'KT-INGLESIDE/THIRD':
        row['ROUTE_SHORT_NAME'] = 'KT'
    elif row['Route'] == 'L-TARAVAL':
        row['ROUTE_SHORT_NAME'] = 'L'
    elif row['Route'] == 'M-OCEAN VIEW':
        row['ROUTE_SHORT_NAME'] = 'M'
    elif row['Route'] == 'N-JUDAH':
        row['ROUTE_SHORT_NAME'] = 'N'
    return row

In [305]:
riders09 = riders09.apply(lambda x : set_short_name09(x), axis = 1)

In [306]:
riders16.head()

,ROUTE,DIRECTION,STOP NAME,STOP ID,STOP SEQ,Time Period,Total Boardings,Total Alightings,Total Train Load,Number of Trips in Time Period,Average Boardings,Average Alightings,Average Train Load,Stop Latitude,Stop Longitude
0,J-Church,INBOUND,San Jose Ave & Geneva Ave,7778,1,7 - Owl,4.4,-,4.4,9,0.5,NaN,0.5,37.721240,-122.44622
1,J-Church,INBOUND,San Jose Ave & Ocean Ave,6275,2,7 - Owl,2.2,-,6.5,9,0.2,NaN,0.7,37.722862,-122.44479
2,J-Church,INBOUND,San Jose Ave & Santa Ynez Ave,6288,3,7 - Owl,2.2,-,8.7,9,0.2,NaN,1,37.725680,-122.44244
3,J-Church,INBOUND,San Jose Ave & Santa Rosa Ave,6284,4,7 - Owl,8.7,-,17.5,9,1.0,NaN,1.9,37.728992,-122.43992
4,J-Church,INBOUND,San Jose Ave/Glen Park Station,4787,5,7 - Owl,2.2,4.4,15.3,9,0.2,0.5,1.7,37.732525,-122.43342


In [307]:
riders16.ROUTE.unique()

array(['J-Church ', 'KT-Ingleside/Third Street ', 'L-Taraval ',
       'M-Ocean View ', 'N-Judah '], dtype=object)

In [308]:
def set_short_name16(row):
    if row['ROUTE'] == 'J-Church ':
        row['ROUTE_SHORT_NAME'] = 'J'
    elif row['ROUTE'] == 'KT-Ingleside/Third Street ':
        row['ROUTE_SHORT_NAME'] = 'KT'
    elif row['ROUTE'] == 'L-Taraval ':
        row['ROUTE_SHORT_NAME'] = 'L'
    elif row['ROUTE'] == 'M-Ocean View ':
        row['ROUTE_SHORT_NAME'] = 'M'
    elif row['ROUTE'] == 'N-Judah ':
        row['ROUTE_SHORT_NAME'] = 'N'
    return row

In [309]:
riders16 = riders16.apply(lambda x : set_short_name16(x),axis = 1)

In [310]:
riders16.ROUTE_SHORT_NAME.unique()

array(['J', 'KT', 'L', 'M', 'N'], dtype=object)

In [311]:
keep = ['ROUTE_SHORT_NAME','SEQ','DIR','STOP_ID','HEADWAY_S','EOL','SOL','STOP_LAT','STOP_LON','DWELL_S','RUNTIME_S','TOTTIME_S','SERVMILES_S','RUNSPEED_S','TOTSPEED_S']

In [312]:
df_keep09 = df09[keep]
df_keep16 = df16[keep]

In [313]:
riders09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 8 columns):
Route               318 non-null object
STOP                318 non-null object
Boardings           318 non-null object
Alightings          318 non-null object
DIR                 318 non-null int64
STOP_ID             318 non-null int64
STOP NAME           318 non-null object
ROUTE_SHORT_NAME    318 non-null object
dtypes: int64(2), object(6)
memory usage: 20.0+ KB


In [314]:
df_keep09.columns

Index(['ROUTE_SHORT_NAME', 'SEQ', 'DIR', 'STOP_ID', 'HEADWAY_S', 'EOL', 'SOL',
       'STOP_LAT', 'STOP_LON', 'DWELL_S', 'RUNTIME_S', 'TOTTIME_S',
       'SERVMILES_S', 'RUNSPEED_S', 'TOTSPEED_S'],
      dtype='object')

In [315]:
grouped09 = df_keep09.groupby(by = ['ROUTE_SHORT_NAME','STOP_ID','DIR'],as_index = False).mean()

In [316]:
grouped09.head()

,ROUTE_SHORT_NAME,STOP_ID,DIR,SEQ,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S
0,F,3092,0,2.0,16.914719,0.0,0.0,37.807407,-122.414125,0.0,467.46,467.46,26.166,3.360000,3.360000
1,F,3095,0,3.0,16.918522,0.0,0.0,37.807841,-122.410810,0.0,437.54,437.54,27.048,3.710000,3.710000
2,F,3308,0,31.0,46.227273,0.0,0.0,37.762810,-122.428500,0.0,66.00,66.00,5.412,4.920000,4.920000
3,F,3311,0,29.0,9.504425,1.0,0.0,37.762520,-122.434866,0.0,222.77,222.77,19.888,5.360000,5.360000
4,F,3311,1,22.0,12.119355,1.0,0.0,37.762520,-122.434866,0.0,82.39,82.39,9.842,7.166667,7.166667


In [317]:
len(grouped09)

439

In [318]:
len(riders09)

318

In [319]:
riders09.STOP_ID = riders09.STOP_ID.convert_objects(convert_numeric = True)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [320]:
merged09_inner = pd.merge(grouped09,riders09,how = 'inner',on = ['ROUTE_SHORT_NAME','DIR','STOP_ID'])

In [321]:
len(merged09_inner)

318

In [322]:
merged09 = pd.merge(grouped09,riders09,how = 'outer',on = ['ROUTE_SHORT_NAME','DIR','STOP_ID'])

In [323]:
merged09 = merged09.sort(columns = ['ROUTE_SHORT_NAME','DIR','SEQ'])

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [324]:
merged09.to_csv('Outer_Join_2009_Rail_Ridership.csv')

In [325]:
merged09.Boardings = merged09.Boardings.convert_objects(convert_numeric = True)
merged09.Alightings = merged09.Alightings.convert_objects(convert_numeric = True)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [326]:
miss_riders = merged09[(np.isnan(merged09['Boardings']))& (np.isnan(merged09['Alightings']))]

In [327]:
len(miss_riders)

130

In [328]:
miss_gtfs = merged09[np.isnan(merged09['TOTSPEED_S'])]

In [241]:
len(miss_gtfs)

0

In [242]:
miss_gtfs.head()

,ROUTE_SHORT_NAME,STOP_ID,DIR,SEQ,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S,Route,STOP,Boardings,Alightings,STOP NAME


In [243]:
len(riders09)

318

In [244]:
miss_riders.to_csv('Stops Missing Ridership Data.csv')

In [245]:
miss_gtfs.to_csv('Stops Missing GTFS Data.csv')

In [246]:
riders09.head()

,Route,STOP,Boardings,Alightings,DIR,STOP_ID,STOP NAME,ROUTE_SHORT_NAME
0,J-CHURCH,Green Div Yard,0,412,0,5448,SAN JOSE AVE & GENEVA AVE,J
1,J-CHURCH,S.Jose&Ocean,3,172,0,6274,San Jose Ave & Ocean Ave,J
2,J-CHURCH,ROW&Liberty,12,95,0,6221,Right Of Way/Liberty St,J
3,J-CHURCH,ROW&21st,13,148,0,6216,Right Of Way/21st St,J
4,J-CHURCH,Glen Park Sta,18,146,0,4788,San Jose Ave/Glen Park Station,J


In [329]:
df09.sort(columns = ['STOP_ID','ROUTE_SHORT_NAME'])

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,MONTH,DOW,AGENCY_ID,ROUTE_SHORT_NAME,DIR,SEQ,NUMDAYS,OBSDAYS,TRIP_STOPS,OBS_TRIP_STOPS,IMP_TRIP_STOPS,WGT_TRIP_STOPS,STOP_ID,ROUTE_LONG_NAME,ROUTE_TYPE,TRIP_HEADSIGN,HEADWAY_S,FARE,STOPNAME,STOPNAME_AVL,STOP_LAT,STOP_LON,EOL,SOL,TIMEPOINT,ARRIVAL_TIME_DEV,DEPARTURE_TIME_DEV,DWELL_S,DWELL,RUNTIME_S,RUNTIME,TOTTIME_S,TOTTIME,SERVMILES_S,SERVMILES,RUNSPEED_S,RUNSPEED,TOTSPEED_S,TOTSPEED,ONTIME5,ON,OFF,LOAD_ARR,LOAD_DEP,PASSMILES,PASSHOURS,WAITHOURS,FULLFARE_REV,PASSDELAY_DEP,PASSDELAY_ARR,RDBRDNGS,DOORCYCLES,WHEELCHAIR,BIKERACK,CAPACITY,VC,CROWDED,CROWDHOURS
1753,2009-10-01,1,SFMTA,F,0,2,22,0.0,147,0,0.0,NaN,3092,MARKET & WHARVES,0,CASTRO,16.914719,2.0,BEACH ST & MASON ST,nan,37.807407,-122.414125,0,0,NaN,NaN,NaN,0.0,NaN,467.46,NaN,467.46,NaN,26.166,NaN,3.36,NaN,3.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2191,2009-11-01,1,SFMTA,F,0,2,20,0.0,147,0,0.0,NaN,3092,MARKET & WHARVES,0,CASTRO,16.914719,2.0,BEACH ST & MASON ST,nan,37.807407,-122.414125,0,0,NaN,NaN,NaN,0.0,NaN,467.46,NaN,467.46,NaN,26.166,NaN,3.36,NaN,3.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,2009-12-01,1,SFMTA,F,0,2,18,0.0,147,0,0.0,NaN,3092,MARKET & WHARVES,0,CASTRO STATION,16.914719,2.0,BEACH ST & MASON ST,nan,37.807407,-122.414125,0,0,NaN,NaN,NaN,0.0,NaN,467.46,NaN,467.46,NaN,26.166,NaN,3.36,NaN,3.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1754,2009-10-01,1,SFMTA,F,0,3,22,0.0,147,0,0.0,NaN,3095,MARKET & WHARVES,0,CASTRO,16.918522,2.0,BEACH ST & STOCKTON ST,nan,37.807841,-122.410810,0,0,NaN,NaN,NaN,0.0,NaN,437.54,NaN,437.54,NaN,27.048,NaN,3.71,NaN,3.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2192,2009-11-01,1,SFMTA,F,0,3,20,0.0,147,0,0.0,NaN,3095,MARKET & WHARVES,0,CASTRO,16.918522,2.0,BEACH ST & STOCKTON ST,nan,37.807841,-122.410810,0,0,NaN,NaN,NaN,0.0,NaN,437.54,NaN,437.54,NaN,27.048,NaN,3.71,NaN,3.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2630,2009-12-01,1,SFMTA,F,0,3,18,0.0,147,0,0.0,NaN,3095,MARKET & WHARVES,0,CASTRO STATION,16.918522,2.0,BEACH ST & STOCKTON ST,nan,37.807841,-122.410810,0,0,NaN,NaN,NaN,0.0,NaN,437.54,NaN,437.54,NaN,27.048,NaN,3.71,NaN,3.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2156,2009-10-01,1,SFMTA,N,1,16,22,0.0,138,0,0.0,NaN,3212,JUDAH,0,CALTRAIN DEPOT,21.291075,2.0,9TH AVE & IRVING ST,nan,37.763888,-122.466245,0,0,NaN,NaN,NaN,0.0,NaN,139.64,NaN,139.64,NaN,16.698,NaN,7.17,NaN,7.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2594,2009-11-01,1,SFMTA,N,1,16,20,0.0,138,0,0.0,NaN,3212,JUDAH,0,CALTRAIN DEPOT,21.291075,2.0,9TH AVE & IRVING ST,nan,37.763888,-122.466245,0,0,NaN,NaN,NaN,0.0,NaN,139.64,NaN,139.64,NaN,16.698,NaN,7.17,NaN,7.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3019,2009-12-01,1,SFMTA,N,1,16,18,0.0,138,0,0.0,NaN,3212,JUDAH,0,DOWNTOWN/CALTRAIN (M-F),18.030597,2.0,9TH AVE & IRVING ST,nan,37.763888,-122.466245,0,0,NaN,NaN,NaN,0.0,NaN,142.84,NaN,142.84,NaN,16.698,NaN,7.01,NaN,7.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,2009-10-01,1,SFMTA,L,0,39,22,0.0,128,0,0.0,NaN,3266,TARAVAL,0,S F ZOO,9.645089,2.0,15TH AVE & TARAVAL ST,nan,37.743124,-122.471297,0,0,NaN,NaN,NaN,0.0,NaN,85.54,NaN,85.54,NaN,14.080,NaN,9.88,NaN,9.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [248]:
df_keep09.sort(columns = ['STOP_ID','ROUTE_SHORT_NAME'])

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,ROUTE_SHORT_NAME,SEQ,DIR,STOP_ID,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S
1753,F,2,0,3092,16.914719,0,0,37.807407,-122.414125,0.0,467.46,467.46,26.166,3.36,3.36
2191,F,2,0,3092,16.914719,0,0,37.807407,-122.414125,0.0,467.46,467.46,26.166,3.36,3.36
2629,F,2,0,3092,16.914719,0,0,37.807407,-122.414125,0.0,467.46,467.46,26.166,3.36,3.36
1754,F,3,0,3095,16.918522,0,0,37.807841,-122.410810,0.0,437.54,437.54,27.048,3.71,3.71
2192,F,3,0,3095,16.918522,0,0,37.807841,-122.410810,0.0,437.54,437.54,27.048,3.71,3.71
2630,F,3,0,3095,16.918522,0,0,37.807841,-122.410810,0.0,437.54,437.54,27.048,3.71,3.71
2156,N,16,1,3212,21.291075,0,0,37.763888,-122.466245,0.0,139.64,139.64,16.698,7.17,7.17
2594,N,16,1,3212,21.291075,0,0,37.763888,-122.466245,0.0,139.64,139.64,16.698,7.17,7.17
3019,N,16,1,3212,18.030597,0,0,37.763888,-122.466245,0.0,142.84,142.84,16.698,7.01,7.01
2001,L,39,0,3266,9.645089,0,0,37.743124,-122.471297,0.0,85.54,85.54,14.080,9.88,9.88


In [174]:
merged09.sort(columns = ['STOP_ID','ROUTE_SHORT_NAME'])

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,ROUTE_SHORT_NAME,STOP_ID,DIR,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S,Route,STOP,Boardings,Alightings,STOP NAME
0,F,3092.0,0,16.914719,0.000000,0.0,37.807407,-122.414125,0.0,467.460000,467.460000,26.166000,3.360000,3.360000,NaN,NaN,NaN,NaN,NaN
1,F,3095.0,0,16.918522,0.000000,0.0,37.807841,-122.410810,0.0,437.540000,437.540000,27.048000,3.710000,3.710000,NaN,NaN,NaN,NaN,NaN
354,N,3212.0,1,20.204249,0.000000,0.0,37.763888,-122.466245,0.0,140.706667,140.706667,16.698000,7.116667,7.116667,N-JUDAH,9th&Irving,NaN,270.0,9th Ave & Irving St
238,L,3266.0,0,9.648438,0.000000,0.0,37.743124,-122.471297,0.0,85.560000,85.560000,14.080000,9.876667,9.876667,L-TARAVAL,15th&Taraval,77.0,255.0,15th Ave & Taraval St
239,L,3267.0,1,16.553128,0.000000,0.0,37.743069,-122.471405,0.0,150.660000,150.660000,16.128000,6.420000,6.420000,L-TARAVAL,15th&Taraval,280.0,77.0,15th Ave & Taraval St
240,L,3268.0,0,9.648237,0.000000,0.0,37.741525,-122.471192,0.0,121.600000,121.600000,19.072000,9.410000,9.410000,L-TARAVAL,15th&Ulloa,22.0,111.0,15th Ave & Ulloa St
241,L,3269.0,1,16.550941,0.000000,0.0,37.741484,-122.471300,0.0,118.300000,118.300000,13.952000,7.080000,7.080000,L-TARAVAL,15th&Ulloa,47.0,13.0,15th Ave & Ulloa St
2,F,3308.0,0,46.227273,0.000000,0.0,37.762810,-122.428500,0.0,66.000000,66.000000,5.412000,4.920000,4.920000,NaN,NaN,NaN,NaN,NaN
3,F,3311.0,0,9.504425,1.000000,0.0,37.762520,-122.434866,0.0,222.770000,222.770000,19.888000,5.360000,5.360000,NaN,NaN,NaN,NaN,NaN
4,F,3311.0,1,12.119355,1.000000,0.0,37.762520,-122.434866,0.0,82.390000,82.390000,9.842000,7.166667,7.166667,NaN,NaN,NaN,NaN,NaN


In [59]:
len(riders09)

323

In [60]:
len(merged09)

189

In [208]:
merged09.STOP_ID.value_counts()

7058    16
5727    15
6997    15
6996    15
5417    15
5731    15
6994    15
6995    15
5419    15
7217    14
6992    12
5728     9
5730     9
5726     9
6991     9
6998     9
6993     7
4510     6
6932     6
5224     6
5225     6
6898     6
4509     6
5237     6
6739     6
6740     6
7145     6
7125     6
5418     6
5234     6
        ..
3358     3
6623     3
6622     3
6625     3
6636     3
5197     3
3603     3
3602     3
3601     3
3600     3
3599     3
5193     3
6626     3
6637     3
6638     3
6635     3
6634     3
6633     3
6632     3
6631     3
6630     3
6629     3
6628     3
6627     3
5120     3
7396     2
7400     2
7162     2
7164     2
7165     2
Name: STOP_ID, dtype: int64

In [249]:
merged09.columns

Index(['ROUTE_SHORT_NAME', 'STOP_ID', 'DIR', 'SEQ', 'HEADWAY_S', 'EOL', 'SOL',
       'STOP_LAT', 'STOP_LON', 'DWELL_S', 'RUNTIME_S', 'TOTTIME_S',
       'SERVMILES_S', 'RUNSPEED_S', 'TOTSPEED_S', 'Route', 'STOP', 'Boardings',
       'Alightings', 'STOP NAME'],
      dtype='object')

In [252]:
merged09_inner.to_csv('E:/Transit-Casa-Alex/MUNI Rail/Input/Rail Stops Performance Data/MUNI_Rail_Stops_2009.csv')

In [330]:
def set_DIR(row):
    if row['DIRECTION'] == 'INBOUND':
        row['DIR'] = 1
    elif row['DIRECTION'] == 'OUTBOUND':
        row['DIR'] = 0
    elif row['DIRECTION'] == 0:
        row['DIR'] = 0
    return row

In [331]:
riders16 = riders16.apply(lambda x : set_DIR(x),axis = 1)

In [332]:
riders16.DIR.unique()

array([1, 0], dtype=int64)

In [333]:
riders16.head()

,ROUTE,DIRECTION,STOP NAME,STOP ID,STOP SEQ,Time Period,Total Boardings,Total Alightings,Total Train Load,Number of Trips in Time Period,Average Boardings,Average Alightings,Average Train Load,Stop Latitude,Stop Longitude,ROUTE_SHORT_NAME,DIR
0,J-Church,INBOUND,San Jose Ave & Geneva Ave,7778,1,7 - Owl,4.4,-,4.4,9,0.5,NaN,0.5,37.721240,-122.44622,J,1
1,J-Church,INBOUND,San Jose Ave & Ocean Ave,6275,2,7 - Owl,2.2,-,6.5,9,0.2,NaN,0.7,37.722862,-122.44479,J,1
2,J-Church,INBOUND,San Jose Ave & Santa Ynez Ave,6288,3,7 - Owl,2.2,-,8.7,9,0.2,NaN,1,37.725680,-122.44244,J,1
3,J-Church,INBOUND,San Jose Ave & Santa Rosa Ave,6284,4,7 - Owl,8.7,-,17.5,9,1.0,NaN,1.9,37.728992,-122.43992,J,1
4,J-Church,INBOUND,San Jose Ave/Glen Park Station,4787,5,7 - Owl,2.2,4.4,15.3,9,0.2,0.5,1.7,37.732525,-122.43342,J,1


In [334]:
riders16.columns 

Index(['ROUTE', 'DIRECTION', 'STOP NAME', 'STOP ID', 'STOP SEQ', 'Time Period',
       ' Total Boardings ', ' Total Alightings ', ' Total Train Load ',
       'Number of Trips in Time Period', ' Average Boardings ',
       ' Average Alightings ', ' Average Train Load ', 'Stop Latitude',
       'Stop Longitude', 'ROUTE_SHORT_NAME', 'DIR'],
      dtype='object')

In [335]:
riders16.columns  = ['ROUTE', 'DIRECTION', 'STOP NAME', 'STOP_ID', 'STOP SEQ', 'Time Period',
       ' Total Boardings ', ' Total Alightings ', ' Total Train Load ',
       'Number of Trips in Time Period', ' Average Boardings ',
       ' Average Alightings ', ' Average Train Load ', 'Stop Latitude',
       'Stop Longitude', 'ROUTE_SHORT_NAME', 'DIR']

In [336]:
df_keep16 = df16[keep]

In [337]:
grouped16 = df_keep16.groupby(by = ['ROUTE_SHORT_NAME','STOP_ID','DIR'],as_index = False).mean()

In [338]:
grouped16.head()

,ROUTE_SHORT_NAME,STOP_ID,DIR,SEQ,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S
0,E,3092,0,2.0,17.028757,0.0,0.0,37.807407,-122.414125,0.0,45.38,45.38,4.576,6.05,6.05
1,E,3095,0,3.0,16.944853,0.0,0.0,37.807841,-122.410810,0.0,63.62,63.62,6.584,6.21,6.21
2,E,3095,1,16.0,16.000000,1.0,0.0,37.807841,-122.410810,0.0,12.00,12.00,0.135,0.68,0.68
3,E,4501,1,14.0,16.956882,0.0,0.0,37.806945,-122.406278,0.0,63.30,63.30,6.448,6.11,6.11
4,E,4502,0,4.0,17.067101,0.0,0.0,37.806629,-122.406030,0.0,66.84,66.84,7.150,6.42,6.42


In [339]:
riders16.head()

,ROUTE,DIRECTION,STOP NAME,STOP_ID,STOP SEQ,Time Period,Total Boardings,Total Alightings,Total Train Load,Number of Trips in Time Period,Average Boardings,Average Alightings,Average Train Load,Stop Latitude,Stop Longitude,ROUTE_SHORT_NAME,DIR
0,J-Church,INBOUND,San Jose Ave & Geneva Ave,7778,1,7 - Owl,4.4,-,4.4,9,0.5,NaN,0.5,37.721240,-122.44622,J,1
1,J-Church,INBOUND,San Jose Ave & Ocean Ave,6275,2,7 - Owl,2.2,-,6.5,9,0.2,NaN,0.7,37.722862,-122.44479,J,1
2,J-Church,INBOUND,San Jose Ave & Santa Ynez Ave,6288,3,7 - Owl,2.2,-,8.7,9,0.2,NaN,1,37.725680,-122.44244,J,1
3,J-Church,INBOUND,San Jose Ave & Santa Rosa Ave,6284,4,7 - Owl,8.7,-,17.5,9,1.0,NaN,1.9,37.728992,-122.43992,J,1
4,J-Church,INBOUND,San Jose Ave/Glen Park Station,4787,5,7 - Owl,2.2,4.4,15.3,9,0.2,0.5,1.7,37.732525,-122.43342,J,1


In [340]:
riders16[' Total Alightings '] = riders16[' Total Alightings '].convert_objects(convert_numeric = True)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [341]:
riders16[' Total Boardings '] = riders16[' Total Boardings '].convert_objects(convert_numeric = True)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [342]:
agg = {'STOP NAME':'first',' Total Boardings ':'sum',' Total Alightings ':'sum','Stop Latitude':'mean','Stop Longitude':'mean'}

In [343]:
riders16_grouped = riders16.groupby(by = ['STOP_ID','ROUTE_SHORT_NAME','DIR'],as_index = False).agg(agg)

In [344]:
riders16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 17 columns):
ROUTE                             2127 non-null object
DIRECTION                         2127 non-null object
STOP NAME                         2127 non-null object
STOP_ID                           2127 non-null int64
STOP SEQ                          2127 non-null int64
Time Period                       2127 non-null object
 Total Boardings                  1850 non-null float64
 Total Alightings                 1855 non-null float64
 Total Train Load                 2127 non-null object
Number of Trips in Time Period    2127 non-null int64
 Average Boardings                1858 non-null float64
 Average Alightings               1862 non-null float64
 Average Train Load               2127 non-null object
Stop Latitude                     2127 non-null float64
Stop Longitude                    2127 non-null float64
ROUTE_SHORT_NAME                  2127 non-null object
DIR       

In [345]:
riders16.head()

,ROUTE,DIRECTION,STOP NAME,STOP_ID,STOP SEQ,Time Period,Total Boardings,Total Alightings,Total Train Load,Number of Trips in Time Period,Average Boardings,Average Alightings,Average Train Load,Stop Latitude,Stop Longitude,ROUTE_SHORT_NAME,DIR
0,J-Church,INBOUND,San Jose Ave & Geneva Ave,7778,1,7 - Owl,4.4,NaN,4.4,9,0.5,NaN,0.5,37.721240,-122.44622,J,1
1,J-Church,INBOUND,San Jose Ave & Ocean Ave,6275,2,7 - Owl,2.2,NaN,6.5,9,0.2,NaN,0.7,37.722862,-122.44479,J,1
2,J-Church,INBOUND,San Jose Ave & Santa Ynez Ave,6288,3,7 - Owl,2.2,NaN,8.7,9,0.2,NaN,1,37.725680,-122.44244,J,1
3,J-Church,INBOUND,San Jose Ave & Santa Rosa Ave,6284,4,7 - Owl,8.7,NaN,17.5,9,1.0,NaN,1.9,37.728992,-122.43992,J,1
4,J-Church,INBOUND,San Jose Ave/Glen Park Station,4787,5,7 - Owl,2.2,4.4,15.3,9,0.2,0.5,1.7,37.732525,-122.43342,J,1


In [346]:
merged16 = pd.merge(riders16_grouped,grouped16,how = 'inner',on = ['STOP_ID','ROUTE_SHORT_NAME','DIR'])

In [347]:
merged16_outter = pd.merge(riders16_grouped,grouped16,how = 'outer',on = ['STOP_ID','ROUTE_SHORT_NAME','DIR'])

In [348]:
len(merged16)

308

In [349]:
len(merged16_outter)

411

In [350]:
len(riders16_grouped)

308

In [351]:
merged16.columns

Index(['STOP_ID', 'ROUTE_SHORT_NAME', 'DIR', 'STOP NAME', 'Stop Latitude',
       ' Total Alightings ', 'Stop Longitude', ' Total Boardings ', 'SEQ',
       'HEADWAY_S', 'EOL', 'SOL', 'STOP_LAT', 'STOP_LON', 'DWELL_S',
       'RUNTIME_S', 'TOTTIME_S', 'SERVMILES_S', 'RUNSPEED_S', 'TOTSPEED_S'],
      dtype='object')

In [352]:
#miss_riders16 = merged16_outter[(np.isnan(merged16_outter[' Average Boardings ']))& (np.isnan(merged16_outter[' Average Alightings ']))]

In [353]:
miss_gtfs16 = merged16_outter[np.isnan(merged16_outter['TOTSPEED_S'])]

In [354]:
len(miss_gtfs16)

0

In [281]:
len(miss_riders16)

106

In [282]:
miss_gtfs16.to_csv('Rail Stops Missing GTFS Data 2016.csv')

In [283]:
miss_riders16.to_csv('Rail Stops Missing Ridership Data 2016.csv')

In [284]:
len(grouped16)

411

In [285]:
len(riders16)

2127

In [286]:
merged16.head(20)

,STOP_ID,ROUTE_SHORT_NAME,DIR,STOP SEQ,Number of Trips in Time Period,Average Boardings,Average Alightings,Stop Latitude,Stop Longitude,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S
0,3212,N,1,112,141,46.8,12.6,264.347384,-857.263680,21.853338,0.0,0.0,37.763913,-122.466241,0.0,132.00,132.00,16.836,7.65,7.65
1,3266,L,0,91,122,5.9,15.5,264.201854,-857.299100,10.237869,0.0,0.0,37.743124,-122.471297,0.0,86.19,86.19,13.420,9.34,9.34
2,3267,L,1,126,122,12.1,4.4,264.201490,-857.299835,17.298743,0.0,0.0,37.743069,-122.471405,0.0,122.91,122.91,15.372,7.50,7.50
3,3268,L,0,84,122,0.9,11.7,264.190668,-857.298330,10.236721,0.0,0.0,37.741525,-122.471192,0.0,125.00,125.00,18.178,8.73,8.73
4,3269,L,1,133,122,4.8,1.1,264.190402,-857.299100,17.299891,0.0,0.0,37.741484,-122.471300,0.0,96.43,96.43,13.298,8.27,8.27
5,3358,M,0,126,104,38.3,127.5,264.047511,-857.326645,11.467925,0.0,0.0,37.721074,-122.475233,0.0,259.27,259.27,43.460,10.06,10.06
6,3361,M,1,77,106,26.3,6.7,264.018230,-857.305925,11.449009,0.0,0.0,37.716889,-122.472275,0.0,148.41,148.41,24.486,9.90,9.90
7,3376,M,0,133,104,13.5,44.9,264.013400,-857.301690,11.467925,0.0,0.0,37.716200,-122.471670,0.0,244.61,244.61,41.234,10.11,10.11
8,3385,M,1,70,106,15.7,3.5,263.999960,-857.287340,11.449858,0.0,0.0,37.714282,-122.469618,0.0,89.10,89.10,15.264,10.28,10.28
9,3403,M,0,119,104,32.9,74.1,264.089112,-857.324580,11.467925,0.0,0.0,37.727017,-122.474937,0.0,181.68,181.68,30.740,10.15,10.15


In [368]:
len(merged16)
len(merged16.STOP_ID.unique())

241

In [282]:
merged16.head()

,STOP_ID,ROUTE_SHORT_NAME,DIR,STOP NAME,Stop Latitude,Total Alightings,Stop Longitude,Total Boardings,SEQ,HEADWAY_S,EOL,SOL,STOP_LAT,STOP_LON,DWELL_S,RUNTIME_S,TOTTIME_S,SERVMILES_S,RUNSPEED_S,TOTSPEED_S
0,3212,N,1,9th Ave & Irving St,37.763912,292.4,-122.466240,1046.3,16.0,21.853338,0.0,0.0,37.763913,-122.466241,0.0,132.00,132.00,16.836,7.65,7.65
1,3266,L,0,15th Ave & Taraval St,37.743122,313.8,-122.471300,119.6,13.0,10.237869,0.0,0.0,37.743124,-122.471297,0.0,86.19,86.19,13.420,9.34,9.34
2,3267,L,1,15th Ave & Taraval St,37.743070,92.3,-122.471405,240.7,18.0,17.298743,0.0,0.0,37.743069,-122.471405,0.0,122.91,122.91,15.372,7.50,7.50
3,3268,L,0,15th Ave & Ulloa St,37.741524,233.4,-122.471190,16.5,12.0,10.236721,0.0,0.0,37.741525,-122.471192,0.0,125.00,125.00,18.178,8.73,8.73
4,3269,L,1,15th Ave & Ulloa St,37.741486,23.3,-122.471300,90.4,19.0,17.299891,0.0,0.0,37.741484,-122.471300,0.0,96.43,96.43,13.298,8.27,8.27


In [280]:
merged16.columns

Index(['STOP_ID', 'ROUTE_SHORT_NAME', 'DIR', 'STOP NAME', 'Stop Latitude',
       ' Total Alightings ', 'Stop Longitude', ' Total Boardings ', 'SEQ',
       'HEADWAY_S', 'EOL', 'SOL', 'STOP_LAT', 'STOP_LON', 'DWELL_S',
       'RUNTIME_S', 'TOTTIME_S', 'SERVMILES_S', 'RUNSPEED_S', 'TOTSPEED_S'],
      dtype='object')

In [283]:
#dropped16 = merged16.drop(['Stop Latitude', 'Stop Longitude',  'Number of Trips in Time Period', 'STOP SEQ'],axis = 1)

In [285]:
merged16.to_csv('E:\Transit-Casa-Alex\MUNI Rail\Input/Rail Stops Performance Data/MUNI_Rail_Stops_2016.csv')

In [390]:
agg = {'STOP NAME':'first', 'STOP_LAT':'mean', 'STOP_LON':'mean'}

In [391]:
merged16_grouped = merged16.groupby(by = 'STOP_ID',as_index = False).agg(agg)

In [392]:
merged16_grouped['geometry'] = [Point(xy) for xy in zip(merged16_grouped.STOP_LON, merged16_grouped.STOP_LAT)]

In [393]:
merged16_geo = gp.GeoDataFrame(merged16_grouped,crs = 'epsg:4326')

In [395]:
merged16_geo.to_file('E:\Transit-Casa-Alex\MUNI Rail\Input/Rail Stops/MUNI_Rail_Stops_2016.shp',driver = 'ESRI Shapefile')

In [374]:
merged09_inner.columns

Index(['ROUTE_SHORT_NAME', 'STOP_ID', 'DIR', 'SEQ', 'HEADWAY_S', 'EOL', 'SOL',
       'STOP_LAT', 'STOP_LON', 'DWELL_S', 'RUNTIME_S', 'TOTTIME_S',
       'SERVMILES_S', 'RUNSPEED_S', 'TOTSPEED_S', 'Route', 'STOP', 'Boardings',
       'Alightings', 'STOP NAME', 'geometry'],
      dtype='object')

In [373]:
agg = {'STOP NAME':'first', 'STOP_LAT':'mean', 'STOP_LON':'mean'}

In [375]:
merged09_grouped = merged09_inner.groupby(by = "STOP_ID",as_index = False).agg(agg)

In [376]:
merged09_grouped['geometry'] = [Point(xy) for xy in zip(merged09_grouped.STOP_LON, merged09_grouped.STOP_LAT)]

In [377]:
merged09_geo = gp.GeoDataFrame(merged09_grouped,crs = 'epsg:4326')

In [378]:
merged09_geo.to_file('E:\Transit-Casa-Alex\MUNI Rail\Input/Rail Stops Performance Data/MUNI_Rail_Stops_2009.shp',driver = 'ESRI Shapefile')

In [379]:
merged09_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 242 entries, 0 to 241
Data columns (total 5 columns):
STOP_ID      242 non-null int64
STOP NAME    242 non-null object
STOP_LON     242 non-null float64
STOP_LAT     242 non-null float64
geometry     242 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 11.3+ KB


In [380]:
merged09_geo.head()

,STOP_ID,STOP NAME,STOP_LON,STOP_LAT,geometry
0,3212,9th Ave & Irving St,-122.466245,37.763888,POINT (-122.466245 37.763888)
1,3266,15th Ave & Taraval St,-122.471297,37.743124,POINT (-122.471297 37.743124)
2,3267,15th Ave & Taraval St,-122.471405,37.743069,POINT (-122.471405 37.743069)
3,3268,15th Ave & Ulloa St,-122.471192,37.741525,POINT (-122.471192 37.741525)
4,3269,15th Ave & Ulloa St,-122.471300,37.741484,POINT (-122.4713 37.741484)


In [381]:
len(merged09_geo)

242

In [389]:
len(merged16_geo)

241

In [388]:
len(merged16_geo.STOP_ID.unique())

241